In [1]:
import boto3
import random
import traceback
import concurrent.futures
import threading
import redis

In [96]:
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0)

In [252]:
def get_random_value(redis_client, key, list_range):
    # print(f'get_random_value({key}, {list_range})')
    # create LUA script which 1) checks if key exist or empty 2) if empty inserts range 3) get random value from the key/set 4) removes key from set and returns it
    lua_script = """
    local key = ARGV[1]
    local range_as_string = ARGV[2]

    -- check if the set is empty
    if redis.call('SCARD', key) == 0 then
        for i in string.gmatch(range_as_string, "%d+") do
            redis.call('SADD', key, i)
        end
    end

    local random_value = redis.call('SRANDMEMBER', key)

    redis.call('SREM', key, random_value)
    return random_value
    """
    # execute LUA script
    return redis_client.eval(lua_script, 0, key, list_range)


In [245]:
val = get_random_value(redis_client, "pfp_bgs", ','.join(map(str, range(1, 6))))
val_redis_pfp_bgs = redis_client.smembers("pfp_bgs")
print(int(val), [int(v) for v in val_redis_pfp_bgs])


get_random_value(pfp_bgs, 1,2,3,4,5)
2 [5, 3, 4, 1]


In [253]:
# run 100 in threads
def getter():
    val = get_random_value(redis_client, "pfp_bgs", ','.join(map(str, range(1, 6))))
    val_redis_pfp_bgs = redis_client.smembers("pfp_bgs")
    return int(val), [int(v) for v in val_redis_pfp_bgs]
results = []
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    future_to_key = {executor.submit(getter): i for i in range(100)}
    for future in concurrent.futures.as_completed(future_to_key):
        key = future_to_key[future]
        try:
            data = future.result()
            results.append(data)
        except Exception as exc:
            print(f'generated an exception: {exc}')
            traceback.print_exc()
for i in results:
    print(f"result: {i}")

result: (2, [5, 3, 4, 1])
result: (2, [5, 3, 1])
result: (4, [5, 4, 1])
result: (5, [5, 4, 1])
result: (1, [])
result: (3, [5, 4, 1])
result: (5, [3])
result: (1, [5, 3, 4, 1])
result: (4, [2, 4, 1])
result: (5, [])
result: (1, [])
result: (4, [5, 1])
result: (5, [])
result: (3, [2, 4, 5, 1])
result: (3, [5, 4, 1])
result: (2, [2, 3, 4, 5])
result: (4, [3, 1])
result: (3, [5, 4, 1])
result: (1, [5, 4, 1])
result: (2, [2, 3, 5, 1])
result: (5, [5, 3, 4, 1])
result: (4, [2, 5])
result: (5, [5, 4, 1])
result: (2, [5, 3, 1])
result: (3, [])
result: (3, [5])
result: (4, [5, 3, 4, 1])
result: (1, [2, 3, 4])
result: (5, [3, 4])
result: (3, [])
result: (2, [])
result: (4, [3])
result: (1, [3, 5])
result: (1, [2, 4, 5])
result: (1, [3])
result: (2, [5, 4, 1])
result: (2, [])
result: (4, [2, 3, 4, 5])
result: (2, [])
result: (2, [5, 1])
result: (3, [2])
result: (5, [2, 4])
result: (5, [2, 4])
result: (5, [3, 4, 5])
result: (4, [1])
result: (2, [5, 4, 1])
result: (4, [5, 3, 4, 1])
result: (1, [5,